# Bokeh streaming example


## 1. Load data to stream from dataset

In [46]:
import numpy as np
from bokeh.client import push_session
from bokeh.embed import autoload_server
from bokeh.plotting import figure, curdoc
import sys
import pandas as pd


def init_data():
    global idx, stock_data
    stock_data = pd.read_csv("data/stock-sample.data")
    idx=0

def next_data():
    global idx, stock_data    
    x = idx
    y = stock_data.iloc[idx]["Close"]
    idx += 1 if idx<len(stock_data) else 0
    return x,y 

init_data()
print stock_data.shape
stock_data.head()

(3842, 7)


,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-04-12,21.530001,22.240000,21.320000,22.049999,196800,22.049999
1,2016-04-11,21.629999,21.950001,21.469999,21.540001,122200,21.540001
2,2016-04-08,21.459999,21.600000,21.340000,21.530001,71400,21.530001
3,2016-04-07,21.450001,21.670000,21.250000,21.379999,54900,21.379999
4,2016-04-06,21.620001,21.709999,21.360001,21.600000,53500,21.600000


## 2. Run the web server

remeber to map ports **5999** and **5006** if using the course virtual machine

In [47]:
%%script bash --bg
kill `ps -efa | grep SimpleHTTPServer | grep python | awk '{print $2}'`
python -m SimpleHTTPServer 5999

Starting job # 15 in a separate thread.


## 3. Run the Bokeh server

In [48]:
%%script bash --bg
kill `ps -efa | grep "bokeh serve" | grep python | awk '{print $2}'`
bokeh serve --allow-websocket-origin=localhost:5999

Starting job # 16 in a separate thread.


## 4. Publish streaming data to the server

do **Kernel->Interrupt** to stop streaming data

In [50]:
cd = curdoc()
cd.clear()

init_data()

x_list = []
y_list = []
for i in range(100):
    d = next_data()
    x_list.append(d[0])
    y_list.append(d[1])

p = figure(plot_width=600, plot_height=300, title="STOCK EXCHANGE TIME SERIES")
r = p.circle(x_list, y_list)
p.y_range.start = 0
p.y_range.end   = 30
# open a session which will keep our local doc in sync with server
session = push_session(curdoc())

html = """
<html>
  <head></head>
  <body>
  <h1>Time Series from STOCK EXCHANGE DATASET</h1> 
  This is a time series <b>MONITORING PANEL</b> with Bokeh streaming
    %s
    
  Please contact <a href='mailto:someone@somewhere'>someone</a> for more information.
  </body>
</html>
""" % autoload_server(p, session_id=session.id)

with open("stock.html", "w+") as f:
    f.write(html)

ds = r.data_source

def update():
    global x_list, y_list
    nd = next_data()
    x_list = x_list[1:] + [nd[0]]
    y_list = y_list[1:] + [nd[1]]
    ds.data.update(x = x_list, y = y_list)
    
cd.add_periodic_callback(update, 500)

print "open the following URL\nhttp://localhost:5999/stock.html"

session.loop_until_closed() # run forever

open the following URL
http://localhost:5999/stock.html


## 5. Stop all servers

In [44]:
!kill `ps -efa | grep SimpleHTTPServer | grep python | awk '{print $2}'`
!kill `ps -efa | grep "bokeh serve" | grep python | awk '{print $2}'`